### Introduction


Throughout our analysis, we will be using a large data of Asteroids - NeoWs. We will discusses the factors that might be usefull to identify an asteroid as potentially hazardous or not. Also we will explains the classification power of those remaining factors in the absence of absolute magnitude and minimum orbit intersection values. 

Near Earth Objects (NEOs) are comets and asteroids that have been nudged by the gravitational attraction of nearby planets into orbits that allow them to enter the Earth’s neighborhood. The scientific interest in comets and asteroids is due largely to their status as the relatively unchanged remnant debris from the solar system formation process some 4.6 billion years ago. As the primitive, leftover building blocks of the solar system formation process, comets and asteroids offer clues to the chemical mixture from which the planets formed some 4.6 billion years ago. 

### Data

The data is about Asteroids - NeoWs. NeoWs (Near Earth Object Web Service) is a RESTful web service for near earth Asteroid information. With NeoWs a user can: search for Asteroids based on their closest approach date to Earth, lookup a specific Asteroid with its NASA JPL small body id, as well as browse the overall data-set.

Acknowledgements Data-set: All the data is from the (http://neo.jpl.nasa.gov/). This API is maintained by SpaceRocks Team: David Greenfield, Arezu Sarvestani, Jason English and Peter Baunach.

### Tasks


* Develop a model that predicts if an asteroid is going to be hazardous or not

To tackle this task I'll use the methods listed below, and extract the results from the method with best performance on unseen data.


In [ ]:
# Libraries & Data import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Data Exploration

To begin this exploratory analysis, first we will check the data. 

In [ ]:
# Data inspection
df = pd.read_csv('nasa.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

There are total of 00 different variables and among them 00 are numerical variables and 00 are Categorical variables.

Neo Reference ID	
Name	
Absolute Magnitude	
Est Dia in KM(min)	
Est Dia in KM(max)	
Est Dia in M(min)	
Est Dia in M(max)	
Est Dia in Miles(min)	
Est Dia in Miles(max)	
Est Dia in Feet(min)	
Est Dia in Feet(max)	
Close Approach Date	
Epoch Date Close Approach	
Relative Velocity km per sec	
Relative Velocity km per hr	
Miles per hour
Miss Dist.(Astronomical)	
Miss Dist.(lunar)	
Miss Dist.(kilometers)	
Miss Dist.(miles)	
Orbiting Body	
Orbit ID	
Orbit Determination Date	
Orbit Uncertainity	
Minimum Orbit Intersection	
Jupiter Tisserand Invariant	
Epoch Osculation	
Eccentricity	
Semi Major Axis	
Inclination	
Asc Node Longitude	
Orbital Period	
Perihelion Distance	
Perihelion Arg	
Aphelion Dist	
Perihelion Time	
Mean Anomaly	
Mean Motion	
Equinox	
Hazardous


Next, use matplotlib to import libraries and define functions for plotting the data.

In [ ]:
# Data visualizaion

### Analysis

In [ ]:
# Categorical relationship

### Data Splitting

In [ ]:
# machine learning

### Conclusion